# 🌱 AI4AD ADM1 Workshop

Welcome to the **Anaerobic Digestion Modelling Workshop**.

This notebook uses the **Anaerobic Digestion Model No. 1 (ADM1)** to simulate:
- Methane production
- Process stability (pH, FOS/TAC)
- Effects of different feedstock mixtures

👉 You don’t need to install anything. Just run each cell in order.

## Step 1: Setup

We start by cloning the GitHub repository `AI4AD-ADM1-Workshop` which contains:
- `ADM1.py` → the simulator code
- `Workshop.ipynb` → this notebook

We also install any required Python libraries.

In [ ]:
!git clone https://github.com/benmola/AI4AD-ADM1-Workshop..git
%cd AI4AD-ADM1-Workshop

!pip install -r requirements.txt

from ADM1 import ADM1Simulator

## Step 2: Feedstock Database

The database contains typical values (from literature) for common UK feedstocks:
- Maize silage
- Grass silage
- Food waste
- Cattle slurry

Each has different proportions of carbohydrates, proteins, lipids, and nitrogen.
This affects methane yield and process stability.

In [ ]:
import pandas as pd

feed_db = pd.read_excel("Feedstock_Characteristics_Database.xlsx")
feed_db.head()

## Step 3: Feeding Plan

You can set the fractions of each feedstock.  
They will **auto-normalise** so total = 1.

You also set:
- HRT (Hydraulic Retention Time)
- Reactor Volume (V)
- Flow rate (Q)
- Temperature (°C)

These values are saved in `Fixed_Feeding.xlsx` and passed to the ADM1 simulator.

In [ ]:
def generate_feeding_excel(feed_plan, filename="Input/Fixed_Feeding.xlsx"):
    feedstocks = ["maize_silage", "grass_silage", "food_waste", "cattle_slurry"]
    total = sum(feed_plan.get(k,0) for k in feedstocks)
    if total > 0:
        feed_plan = {k: feed_plan.get(k,0)/total for k in feedstocks}
    df = pd.DataFrame([{
        "time": 1,
        "maize_silage": feed_plan.get("maize_silage",0),
        "grass_silage": feed_plan.get("grass_silage",0),
        "food_waste": feed_plan.get("food_waste",0),
        "cattle_slurry": feed_plan.get("cattle_slurry",0),
        "HRT": feed_plan.get("HRT",48),
        "V": feed_plan.get("V",6520),
        "Q": feed_plan.get("Q",136.63),
        "T": feed_plan.get("T",45)
    }])
    df.to_excel(filename, index=False)
    return filename

## Step 4: Running the ADM1 Model

Now we simulate the digester for the chosen feedstock mix and parameters.

The model outputs:
- Methane flow
- Biogas
- pH
- FOS/TAC ratio
- Gas pressure

You’ll see both **plots** and a **summary table**.

In [ ]:
import matplotlib.pyplot as plt

def run_adm1(maize_silage, grass_silage, food_waste, cattle_slurry, HRT, V, Q, T):
    feed_plan = {
        "maize_silage": maize_silage,
        "grass_silage": grass_silage,
        "food_waste": food_waste,
        "cattle_slurry": cattle_slurry,
        "HRT": HRT,
        "V": V,
        "Q": Q,
        "T": T
    }
    feed_file = generate_feeding_excel(feed_plan)
    simulator = ADM1Simulator("Feedstock_Characteristics_Database.xlsx", feed_file)
    simulator.run()
    _, output_data = simulator.get_results()
    if output_data.empty:
        print("⚠️ No results returned.")
        return
    summary = {
        "pH": output_data["pH"].iloc[-1],
        "FOS": output_data["FOS"].iloc[-1],
        "TAC": output_data["TAC"].iloc[-1],
        "FOS/TAC": output_data["FOS/TAC"].iloc[-1],
        "Methane Flow Rate": output_data["q_ch4"].iloc[-1],
        "Biogas": output_data["q_gas"].iloc[-1],
        "Gas Pressure": output_data["P_gas"].iloc[-1]
    }
    fig, ax = plt.subplots(3,1, figsize=(8,10), sharex=True)
    ax[0].plot(output_data["time"], output_data["q_ch4"])
    ax[0].set_ylabel("Methane (m³/d)")
    ax[1].plot(output_data["time"], output_data["pH"], color="green")
    ax[1].set_ylabel("pH")
    ax[2].plot(output_data["time"], output_data["FOS"], color="red", label="FOS")
    ax[2].plot(output_data["time"], output_data["TAC"], color="blue", label="TAC")
    ax[2].set_ylabel("FOS/TAC")
    ax[2].legend()
    ax[2].set_xlabel("Time (days)")
    plt.show()
    print("📊 Simulation summary:")
    for k,v in summary.items():
        print(f"- {k}: {v:.2f}")

## Step 5: Interactive Sliders

Use the sliders to experiment with feed fractions and operating conditions.  
Try scenarios like:
- All maize silage
- All food waste
- Short vs long HRT

👉 The goal is to observe **trends**, not exact numbers.

In [ ]:
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact

interact(
    run_adm1,
    maize_silage=FloatSlider(min=0, max=1, step=0.1, value=0.5, description="Maize Silage"),
    grass_silage=FloatSlider(min=0, max=1, step=0.1, value=0.3, description="Grass Silage"),
    food_waste=FloatSlider(min=0, max=1, step=0.1, value=0.1, description="Food Waste"),
    cattle_slurry=FloatSlider(min=0, max=1, step=0.1, value=0.1, description="Cattle Slurry"),
    HRT=FloatSlider(min=10, max=80, step=2, value=48, description="HRT (days)"),
    V=FloatSlider(min=1000, max=10000, step=500, value=6520, description="V (m³)"),
    Q=FloatSlider(min=50, max=500, step=10, value=136.63, description="Q (m³/d)"),
    T=FloatSlider(min=15, max=60, step=1, value=45, description="T (°C)"),
);

## Step 6: Export Results

After running a simulation, export the results as a CSV for further analysis in Excel or other tools.

In [ ]:
from google.colab import files

def export_results(maize_silage=0.5, grass_silage=0.3, food_waste=0.1, cattle_slurry=0.1,
                   HRT=48, V=6520, Q=136.63, T=45):
    feed_plan = {
        "maize_silage": maize_silage,
        "grass_silage": grass_silage,
        "food_waste": food_waste,
        "cattle_slurry": cattle_slurry,
        "HRT": HRT,
        "V": V,
        "Q": Q,
        "T": T
    }
    feed_file = generate_feeding_excel(feed_plan)
    simulator = ADM1Simulator("Feedstock_Characteristics_Database.xlsx", feed_file)
    _, output_data = simulator.get_results()
    filename = "ADM1_results.csv"
    output_data.to_csv(filename, index=False)
    files.download(filename)
    print(f"✅ CSV exported: {filename}")

# ✅ Wrap-Up

Congratulations! You’ve run your first ADM1 simulations in Python.

- Different feedstocks → different methane yields and process stability  
- Process conditions (HRT, volume, flow, temperature) → influence stability and efficiency  

📥 You can re-run this anytime via the [GitHub repo](https://github.com/benmola/AI4AD-ADM1-Workshop.).

🙋 Questions? Discuss with your facilitator or open an issue in the GitHub repo.